In [1]:
# Cài đặt nếu chưa có (bỏ comment nếu cần)

!pip install pytorch-fid lpips


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.5 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 6.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.0 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvi

In [3]:
import shutil
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torchvision.utils import save_image
from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image
import os
import numpy as np
from tqdm import tqdm
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
from pytorch_fid import fid_score
import lpips
import matplotlib.pyplot as plt
from skimage.color import lab2rgb, rgb2lab
import json
import zipfile
import tempfile
from torchvision import transforms as tv_transforms

In [4]:
# Thiết bị
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Cấu hình
image_size = 256
batch_size = 16
lr = 0.0001
epochs = 50
INPUT_IMAGE_CHANNELS = 1
GEN_IMAGE_CHANNELS = 2  # Output ab channels

save_dir = '/kaggle/working/unet_basic_output'
os.makedirs(save_dir, exist_ok=True)
save_bad = '/kaggle/working/unet_bad_images'
os.makedirs(save_bad, exist_ok=True)
img_dir = '/kaggle/input/daaataaaa/data1'

checkpoint_dir = '/kaggle/working/unet_checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)
start_epoch = 0
best_loss = float('inf')
best_score = -float('inf')

Using device: cuda


In [5]:
valid_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.tif', '.gif')
error_count = 0

for root, dirs, files in os.walk(img_dir):
    for file in files:
        if file.lower().endswith(valid_extensions):
            path = os.path.join(root, file)
            try:
                with Image.open(path) as img:
                    img.verify()
            except Exception as e:
                error_count += 1
                print(f"❌ Ảnh lỗi: {path} ({e})")
                shutil.move(path, os.path.join(save_bad, file))

print(f"✅ Đã xử lý xong: {error_count} ảnh lỗi đã được di chuyển vào {save_bad}")
class ColorizationDataset(Dataset):
    def __init__(self, root_dir, img_size=256, is_train=True):   # 256 thành 128
        self.root_dir = root_dir
        self.img_size = img_size
        self.is_train = is_train
        self.image_paths = []

        # Thu thập tất cả các đường dẫn ảnh trong thư mục gốc và các thư mục con
        for root, _, files in os.walk(root_dir):
            for file in files:
                if file.lower().endswith(valid_extensions): # Sử dụng valid_extensions đã định nghĩa
                    self.image_paths.append(os.path.join(root, file))

        # Định nghĩa các phép biến đổi ảnh
        if self.is_train:
            self.transforms = transforms.Compose([
                transforms.Resize(self.img_size + 30), # Resize lớn hơn một chút  # chỉnh 30 về 20 của 128
                transforms.RandomCrop(self.img_size),  # Cắt ngẫu nhiên
                transforms.RandomHorizontalFlip(),      # Lật ngang 
                transforms.RandomRotation(10),          # Xoay ngẫu nhiên   từ 10 về 5   của 128
                transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.05), # Cải thiện ColorJitter
                transforms.ToTensor(),                  # Chuyển đổi sang Tensor (RGB, [0, 1])
                
            ])
        else: # Đối với validation/test, chỉ cần resize và chuẩn hóa
            self.transforms = transforms.Compose([
                transforms.Resize((self.img_size, self.img_size)),
                transforms.ToTensor(),
               
            ])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        try:
            # Mở ảnh và chuyển sang RGB
            img_rgb_pil = Image.open(img_path).convert('RGB')
            
            # Áp dụng transforms cho ảnh RGB (PIL Image)
            img_rgb_tensor = self.transforms(img_rgb_pil) 
            
            # Chuyển đổi từ RGB Tensor về numpy array để sang L*a*b*
            # skimage.rgb2lab mong đợi numpy array với giá trị trong khoảng [0, 255]
            img_rgb_np = img_rgb_tensor.permute(1, 2, 0).numpy() # (C, H, W) -> (H, W, C)
            #img_rgb_np = (img_rgb_np * 0.5 + 0.5) * 255 # Đưa về khoảng [0, 255] nhân hai lần 255 khiến ảnh sáng, làm học sai
             # Chuẩn hóa đúng cách từ [0, 1] về [0, 255] và ép kiểu
            img_rgb_np = (img_rgb_np * 255).astype(np.uint8) # <-- Dòng đã sửa
            
            # Chuyển đổi sang không gian màu L*a*b*
            img_lab = rgb2lab(img_rgb_np).astype("float32") 
            
            # Tách kênh L và kênh a, b, sau đó chuẩn hóa về [-1, 1]
            L = img_lab[:, :, 0] / 50.0 - 1.0 # Kênh L: [0, 100] -> [-1, 1]
            ab = img_lab[:, :, 1:] / 128.0   # Kênh a, b: [-128, 128] -> [-1, 1]
            
            # Chuyển numpy array về Tensor và định dạng đúng (C, H, W)
            L = torch.from_numpy(L).unsqueeze(0) # (1, H, W)
            ab = torch.from_numpy(ab).permute(2, 0, 1) # (2, H, W)

            return L, ab # Trả về kênh L (grayscale) và kênh ab (color)

        except Exception as e:
            print(f"❌ Bỏ qua ảnh lỗi khi load (tại getitem): {img_path} ({e}). Trả về dummy tensors.")
            # Trả về dummy tensors với kích thước chính xác cho IMG_SIZE
            dummy_L = torch.zeros((1, self.img_size, self.img_size))
            dummy_ab = torch.zeros((2, self.img_size, self.img_size))
            return dummy_L, dummy_ab



dataset = ColorizationDataset(
    root_dir=img_dir,
    img_size=image_size,
    is_train=True # is_train chỉ ảnh hưởng đến augmentations, không ảnh hưởng đến việc chia dataset
)
train_dataset, test_dataset = random_split(dataset, [int(0.8 * len(dataset)), len(dataset) - int(0.8 * len(dataset))])
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print("Train size:", len(train_dataset), "Test size:", len(test_dataset)) 

✅ Đã xử lý xong: 0 ảnh lỗi đã được di chuyển vào /kaggle/working/unet_bad_images
Train size: 1835 Test size: 459


In [6]:
sample_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

for L, ab in sample_loader:
    print("L shape:", L.shape)
    print("ab shape:", ab.shape)
    break


L shape: torch.Size([16, 1, 256, 256])
ab shape: torch.Size([16, 2, 256, 256])


In [13]:
import torch
import torch.nn as nn

class UnetBasic1(nn.Module):
    def __init__(self, input_channels=1, output_channels=2):
        super(UnetBasic1, self).__init__()

        # Encoder (ít tầng, ít kênh)
        self.enc1 = self.contract_block(input_channels, 16, 4, 2, 1)  # 128 -> 64
        self.enc2 = self.contract_block(16, 32, 4, 2, 1)              # 64 -> 32
        self.enc3 = self.contract_block(32, 64, 4, 2, 1)              # 32 -> 16

        # Bottleneck
        self.middle = nn.Sequential(
            nn.Conv2d(64, 128, 3, 1, 1),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2)
        )

        # Decoder (ít tầng)
        self.dec3 = self.expand_block(128+64, 64, 4, 2, 1)   # 16 -> 32
        self.dec2 = self.expand_block(64+32, 32, 4, 2, 1)    # 32 -> 64
        self.dec1 = self.expand_block(32+16, 16, 4, 2, 1)    # 64 -> 128

        # Output
        self.final = nn.Sequential(
            nn.Conv2d(16, output_channels, 3, 1, 1),
            nn.Tanh()
        )

    def contract_block(self, in_channels, out_channels, kernel_size, stride, padding):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.2, inplace=True)
        )

    def expand_block(self, in_channels, out_channels, kernel_size, stride, padding):
        return nn.Sequential(
            nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride, padding),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        e1 = self.enc1(x)
        e2 = self.enc2(e1)
        e3 = self.enc3(e2)

        m = self.middle(e3)

        d3 = self.dec3(torch.cat([m, e3], dim=1))
        d2 = self.dec2(torch.cat([d3, e2], dim=1))
        d1 = self.dec1(torch.cat([d2, e1], dim=1))

        return self.final(d1)


In [9]:
import warnings
# Tùy chọn: ẩn cảnh báo "negative Z values" từ skimage
warnings.filterwarnings("ignore", message="Conversion from CIE-LAB.*", category=UserWarning)

import numpy as np
import torch
from skimage.color import lab2rgb

def lab_to_rgb_tensor(L_channel, ab_channel):
    """
    Chuyển đổi từ tensor LAB sang tensor RGB.
    
    Args:
        L_channel (torch.Tensor): Kênh L (lightness), giá trị trong khoảng [-1, 1]. Shape: (B, 1, H, W)
        ab_channel (torch.Tensor): Kênh a và b, giá trị trong khoảng [-1, 1]. Shape: (B, 2, H, W)
        device: 'cuda' hoặc 'cpu'

    Returns:
        torch.Tensor: Tensor RGB, giá trị trong khoảng [-1, 1]. Shape: (B, 3, H, W)
    """
    device = L_channel.device  # Lấy device từ input
    # 1. Chuyển đổi tensor về numpy
    L_np = L_channel.detach().cpu().numpy()
    ab_np = ab_channel.detach().cpu().numpy()
    
    # 2. Chuẩn hóa ngược về các giá trị LAB gốc
    # L trong không gian LAB chuẩn có giá trị từ [0, 100]
    # a, b trong không gian LAB chuẩn có giá trị từ [-128, 128]
    L_np = (L_np + 1.0) / 2.0 * 100.0  # Chuẩn hóa ngược kênh L từ [-1, 1] về [0, 100]
    ab_np = ab_np * 128.0 # Chuẩn hóa ngược kênh ab từ [-1, 1] về [-128, 128]
    
    # 3. Kết hợp các kênh
    lab_np = np.concatenate((L_np, ab_np), axis=1) # (B, 3, H, W)
    lab_np = lab_np.transpose((0, 2, 3, 1)) # (B, H, W, 3)

    rgb_imgs = []
    for img_lab in lab_np:
        # img_lab có shape (H, W, 3)
        img_rgb = lab2rgb(img_lab)  # trả về ảnh RGB có dải giá trị [0, 1]
        rgb_imgs.append(img_rgb)
    
    # 4. Chuyển lại về tensor và chuẩn hóa
    rgb_imgs = np.stack(rgb_imgs, axis=0) # (B, H, W, 3)
    rgb_imgs = torch.from_numpy(rgb_imgs).permute(0, 3, 1, 2) # (B, 3, H, W)
    
    # Chuẩn hóa RGB từ [0, 1] về [-1, 1] để phù hợp với mô hình
    rgb_imgs = rgb_imgs * 2.0 - 1.0

    return rgb_imgs.to(device)

In [11]:
# Checkpoint functions
def save_checkpoint(epoch, model, optimizer, best_loss, best_score):
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'best_loss': best_loss,
        'best_score': best_score
    }
    torch.save(checkpoint, os.path.join(checkpoint_dir, "checkpoint_latest.pth"))
    print(f"✅ Đã lưu checkpoint tại epoch {epoch + 1}")

def load_checkpoint(model, optimizer):
    path = os.path.join(checkpoint_dir, "checkpoint_latest.pth")
    if os.path.exists(path):
        checkpoint = torch.load(path, weights_only=False)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        start_epoch = checkpoint['epoch'] + 1
        best_loss = checkpoint['best_loss']
        best_score = checkpoint.get('best_score', -float('inf'))
        print(f"✅ Đã tải checkpoint. Tiếp tục từ epoch {start_epoch}")
        return start_epoch, best_loss, best_score
    else:
        print("⚠️ Không tìm thấy checkpoint. Bắt đầu từ đầu.")
        return 0, float('inf'), -float('inf')

In [ ]:
# Zip folder
def zip_folder(folder_path, zip_path):
    with zipfile.ZipFile(zip_path, 'w') as zipf:
        for root, _, files in os.walk(folder_path):
            for file in files:
                filepath = os.path.join(root, file)
                arcname = os.path.relpath(filepath, folder_path)
                zipf.write(filepath, arcname)

# Training
model = UnetBasic1().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.0001, betas=(0.5, 0.999))
l1_loss_fn = nn.L1Loss()

resume_training = True
if resume_training:
    start_epoch, best_loss, best_score = load_checkpoint(model, optimizer)
else:
    print("🚀 Đang khởi động training từ đầu...")

log_file_path = os.path.join(save_dir, "training_log.txt")
log_file = open(log_file_path, "a" if resume_training else "w")
losses = []
PSNR_scores, SSIM_scores, LPIPS_scores, FID_scores = [], [], [], []
best_epoch = -1
lpips_fn = lpips.LPIPS(net='alex').to(device)  
for epoch in range(start_epoch, epochs):
    loss_epoch = 0
    psnr_total, ssim_total, lpips_total = 0, 0, 0
    n_samples = 0
    loop = tqdm(train_loader, desc=f"Epoch [{epoch + 1}/{epochs}]")

    

    for L_channel_real, ab_channel_real in loop:
        L_channel_real, ab_channel_real = L_channel_real.to(device), ab_channel_real.to(device)

        optimizer.zero_grad()
        fake_ab_channel = model(L_channel_real)
        #print(f"fake_ab_channel min/max: {fake_ab_channel.min().item()}, {fake_ab_channel.max().item()}")
       # print(f"fake_ab_channel mean: {fake_ab_channel.mean().item()}")
        
        reconstruction_loss = l1_loss_fn(fake_ab_channel, ab_channel_real)
        
        real_rgb_for_lpips = lab_to_rgb_tensor(L_channel_real, ab_channel_real).float()
        fake_rgb_for_lpips = lab_to_rgb_tensor(L_channel_real, fake_ab_channel).float()
        perceptual_loss = lpips_fn(fake_rgb_for_lpips, real_rgb_for_lpips).mean() 
        
        loss = 10 * reconstruction_loss + 20 * perceptual_loss
        loss.backward()
        optimizer.step()

        # Tính metrics
        real_rgb = lab_to_rgb_tensor(L_channel_real, ab_channel_real).float()
        fake_rgb = lab_to_rgb_tensor(L_channel_real, fake_ab_channel).float()
        perceptual_loss = lpips_fn(fake_rgb, real_rgb).mean()

        for i in range(real_rgb.size(0)):
            gt_rgb = lab_to_rgb_tensor(L_channel_real[i:i+1], ab_channel_real[i:i+1]).squeeze(0)
            pred_rgb = lab_to_rgb_tensor(L_channel_real[i:i+1], fake_ab_channel[i:i+1]).squeeze(0)
            gt_np = (gt_rgb.detach().cpu().permute(1, 2, 0).numpy() + 1) / 2
            pred_np = (pred_rgb.detach().cpu().permute(1, 2, 0).numpy() + 1) / 2
            psnr_total += psnr(gt_np, pred_np, data_range=1)
            ssim_total += ssim(gt_np, pred_np, data_range=1, channel_axis=-1)

        lpips_total += perceptual_loss.item() * real_rgb.size(0)
        n_samples += real_rgb.size(0)

        loop.set_postfix(Loss=loss.item())
        loss_epoch += loss.item()

    # Tính FID
    temp_real_dir = tempfile.mkdtemp()
    temp_gen_dir = tempfile.mkdtemp()

    test_L = L_channel_real[:128].detach()
    test_ab_fake = fake_ab_channel[:128].detach()

    for i in range(test_L.size(0)):
        real_rgb = lab_to_rgb_tensor(test_L[i:i+1], ab_channel_real[i:i+1]).squeeze(0)
        fake_rgb = lab_to_rgb_tensor(test_L[i:i+1], test_ab_fake[i:i+1]).squeeze(0)
        real_rgb = (real_rgb + 1) / 2
        fake_rgb = (fake_rgb + 1) / 2
        tv_transforms.ToPILImage()(real_rgb).save(f"{temp_real_dir}/{i}.png")
        tv_transforms.ToPILImage()(fake_rgb).save(f"{temp_gen_dir}/{i}.png")

    fid = fid_score.calculate_fid_given_paths([temp_real_dir, temp_gen_dir], batch_size=16, device=device, dims=2048)

    # Cập nhật metrics
    avg_loss = loss_epoch / len(train_loader)
    psnr_avg = psnr_total / n_samples
    ssim_avg = ssim_total / n_samples
    lpips_avg = lpips_total / n_samples

    losses.append(avg_loss)
    PSNR_scores.append(psnr_avg)
    SSIM_scores.append(ssim_avg)
    LPIPS_scores.append(lpips_avg)
    FID_scores.append(fid)

    # Ghi log
    log_file.write(
        f"Epoch {epoch + 1:03d}:\n"
        f"  Loss:    {avg_loss:.4f}\n"
        f"  PSNR:    {psnr_avg:.2f}\n"
        f"  SSIM:    {ssim_avg:.4f}\n"
        f"  LPIPS:   {lpips_avg:.4f}\n"
        f"  FID:     {fid:.4f}\n"
        "-------------------------\n"
    )
    log_file.flush()
    # Lưu ảnh kết quả
    fake_rgb_to_save = lab_to_rgb_tensor(test_L, test_ab_fake)
    ##print("Fake RGB tensor stats:", fake_rgb_to_save.min().item(), fake_rgb_to_save.max().item())
    # Chuyển từ [-1, 1] về [0, 1]  để hiển thị ảnh vì hàm lab_to_rgb_tensor đang ở dải [-1, 1]
    fake_rgb_to_save = (fake_rgb_to_save + 1.0) / 2.0 
    save_image(
    (fake_rgb_to_save[:30]),
    os.path.join(save_dir, f"epoch_{epoch + 1}.png"),
    nrow=5,
    normalize=False  # hoặc bỏ normalize luôn
)
    

    # Lưu mô hình tốt nhất
    score = -10 * lpips_avg + 5.0 * ssim_avg + psnr_avg
    if score > best_score:
        best_score = score
        best_epoch = epoch + 1
        torch.save(model.state_dict(), os.path.join(save_dir, 'model_best.pth'))
        print(f"✅ Đã lưu mô hình tốt nhất tại epoch {best_epoch} với PSNR = {psnr_avg:.4f}, LPIPS = {lpips_avg:.4f}, SSIM = {ssim_avg:.4f}, FID = {fid:.4f}, Score = {best_score:.4f}")

    save_checkpoint(epoch, model, optimizer, best_loss, best_score)

    if (epoch + 1) % 10 == 0:
        zip_folder(save_dir, f'/kaggle/working/unet_output_epoch{epoch + 1}.zip')

    if (epoch + 1) % 20 == 0:
        zip_folder(checkpoint_dir, f'/kaggle/working/unet_checkpoints_epoch{epoch + 1}.zip')

# Lưu mô hình cuối cùng
torch.save(model.state_dict(), os.path.join(save_dir, 'model_final.pth'))
log_file.close()

⚠️ Không tìm thấy checkpoint. Bắt đầu từ đầu.
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:01<00:00, 166MB/s]  


Loading model from: /usr/local/lib/python3.11/dist-packages/lpips/weights/v0.1/alex.pth


Epoch [1/50]: 100%|██████████| 115/115 [03:15<00:00,  1.70s/it, Loss=4.11]
Downloading: "https://github.com/mseitzer/pytorch-fid/releases/download/fid_weights/pt_inception-2015-12-05-6726825d.pth" to /root/.cache/torch/hub/checkpoints/pt_inception-2015-12-05-6726825d.pth
100%|██████████| 91.2M/91.2M [00:00<00:00, 281MB/s]


100%|██████████| 1/1 [00:00<00:00,  3.71it/s]


...

Epoch [38/50]: 100%|██████████| 115/115 [03:03<00:00,  1.60s/it, Loss=5.88]


100%|██████████| 1/1 [00:00<00:00,  5.14it/s]


100%|██████████| 1/1 [00:00<00:00,  5.21it/s]


✅ Đã lưu mô hình tốt nhất tại epoch 38 với PSNR = 23.7251, LPIPS = 0.2037, SSIM = 0.9229, FID = 200.3472, Score = 26.3026
✅ Đã lưu checkpoint tại epoch 38


Epoch [39/50]:  43%|████▎     | 50/115 [01:21<01:44,  1.60s/it, Loss=5.32]

In [ ]:
# Đánh giá trên test set
def evaluate_model(model, dataloader, real_dir, gen_dir):
    os.makedirs(real_dir, exist_ok=True)
    os.makedirs(gen_dir, exist_ok=True)

    lpips_fn = lpips.LPIPS(net='alex').to(device)
    to_pil = tv_transforms.ToPILImage()

    def denormalize(tensor):
        return tensor * 0.5 + 0.5

    psnr_list, ssim_list, lpips_list = [], [], []

    model.eval()
    with torch.no_grad():
        for i, (L_channel, ab_channel) in enumerate(tqdm(dataloader)):
            L_channel, ab_channel = L_channel.to(device), ab_channel.to(device)
            fake_ab = model(L_channel)

            for j in range(L_channel.size(0)):
                real_rgb = lab_to_rgb_tensor(L_channel[j:j+1], ab_channel[j:j+1]).squeeze(0)
                fake_rgb = lab_to_rgb_tensor(L_channel[j:j+1], fake_ab[j:j+1]).squeeze(0)

                real_np = denormalize(real_rgb).cpu().permute(1, 2, 0).numpy()
                fake_np = denormalize(fake_rgb).cpu().permute(1, 2, 0).numpy()

                psnr_list.append(psnr(real_np, fake_np, data_range=1))
                ssim_list.append(ssim(real_np, fake_np, data_range=1, channel_axis=-1))
                lpips_val = lpips_fn(real_rgb.unsqueeze(0).to(device), fake_rgb.unsqueeze(0).to(device)).item()
                lpips_list.append(lpips_val)

                to_pil(denormalize(real_rgb)).save(f"{real_dir}/real_{i}_{j}.png")
                to_pil(denormalize(fake_rgb)).save(f"{gen_dir}/gen_{i}_{j}.png")

    fid = fid_score.calculate_fid_given_paths([real_dir, gen_dir], batch_size=50, device=device, dims=2048)

    psnr_mean = np.mean(psnr_list)
    ssim_mean = np.mean(ssim_list)
    lpips_mean = np.mean(lpips_list)

    metrics_path = os.path.join(save_dir, "metrics_result.txt")
    with open(metrics_path, "w") as f:
        f.write(f"PSNR: {psnr_mean:.4f}\n")
        f.write(f"SSIM: {ssim_mean:.4f}\n")
        f.write(f"LPIPS: {lpips_mean:.4f}\n")
        f.write(f"FID: {fid:.4f}\n")

    with zipfile.ZipFile(os.path.join(save_dir, "metrics_result.zip"), "w") as zipf:
        zipf.write(metrics_path, arcname="metrics_result.txt")

    for dir_path, zip_path in [(real_dir, os.path.join(save_dir, "real_images.zip")), (gen_dir, os.path.join(save_dir, "generated_images.zip"))]:
        zip_folder(dir_path, zip_path)

    return {
        "PSNR": psnr_mean,
        "SSIM": ssim_mean,
        "LPIPS": lpips_mean,
        "FID": fid,
        "psnr_list": psnr_list,
        "ssim_list": ssim_list,
        "lpips_list": lpips_list
    }

real_dir = os.path.join(save_dir, 'real_images')
gen_dir = os.path.join(save_dir, 'generated_images')
model.load_state_dict(torch.load(os.path.join(save_dir, 'model_best.pth')))
metrics = evaluate_model(model, test_loader, real_dir, gen_dir)
print(metrics)

In [ ]:
# Vẽ biểu đồ
plt.figure(figsize=(8, 5))
plt.plot(range(1, len(losses) + 1), losses, label="L1 Loss", color='blue')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("L1 Loss over Epochs")
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(save_dir, "loss_over_epochs.png"))
plt.show()

plt.figure(figsize=(8, 5))
plt.plot(range(1, len(PSNR_scores) + 1), PSNR_scores, marker='o', color='green', label='PSNR')
plt.xlabel("Epoch")
plt.ylabel("PSNR")
plt.title("PSNR over Epochs")
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(save_dir, "psnr_over_epochs.png"))
plt.show()

plt.figure(figsize=(8, 5))
plt.plot(range(1, len(SSIM_scores) + 1), SSIM_scores, marker='o', color='orange', label='SSIM')
plt.xlabel("Epoch")
plt.ylabel("SSIM")
plt.title("SSIM over Epochs")
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(save_dir, "ssim_over_epochs.png"))
plt.show()

plt.figure(figsize=(8, 5))
plt.plot(range(1, len(LPIPS_scores) + 1), LPIPS_scores, marker='o', color='purple', label='LPIPS')
plt.xlabel("Epoch")
plt.ylabel("LPIPS")
plt.title("LPIPS over Epochs")
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(save_dir, "lpips_over_epochs.png"))
plt.show()

plt.figure(figsize=(8, 5))
plt.plot(range(1, len(FID_scores) + 1), FID_scores, label='FID', color='brown')
plt.title('FID per Epoch')
plt.xlabel('Epoch')
plt.ylabel('FID')
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(save_dir, 'fid_curve.png'))
plt.show()